# Nearest Neighbour (item-based)

In this notebook, nearest neighbor algorithm will be implemented to create a recommendation system for movies. 

This notebook contains part of the documentation. The full details of the algorithm can be found in the accompanying report found in the repository.

First, we will install the dependencies. We need to install numpy, sklearn and scipy

In [1]:
%pip install numpy scipy sklearn

Note: you may need to restart the kernel to use updated packages.


Now we import the required dependencies.

In [2]:
import math
import operator
import multiprocessing
import time
import json
import numpy as np
from typing import Any, Dict, List
from multiprocessing import Process
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

In order to retrieve the train, test and validation datasets as well as the constructed utility matrix, we will the MatrixMaker class.
The class is responsible for constructing the matrices and storing them to save computation time. Make sure to delete this files when changing the ratings file.
The full functionality of the class can be found in the documentation.

In [3]:
from matrix_maker import MatrixMaker

data_retriever = MatrixMaker()
(train_set, test_set, validation_set, utility_matrix) = data_retriever.make_matrices(remake=False)

Files found, returning the matrices.


We will use the Logger class to log meta data about the execution of the program so it can be used later to analyse the algorithm.

In [4]:
from logger import Logger

OPERATION = "nearest_neighbour"
logger = Logger(OPERATION)

We will get the program configuration from a configuration file. This is done to separate the parameters of the program from the program itself.

In [5]:
config = None
try:
    config = json.load(open("config.json", "r"))
except Exception as e:
    print(e.__doc__)
    print("Check if config file exists and is in good order")

The **predict_nearestneighnor_mp** function uses collaborative filtering to predicted the ratings of the test set.

In [6]:
def predict_nearestneighbor_mp(matrix: np.ndarray, predictions: Any, knn: int, return_dict: Any, pid: int, global_average: float, users_bias: Dict, movies_bias: Dict) -> None:
    """Use nearest neighbor collaborative filtering to predict ratings. Built with multiprocessing in mind.

    Args:
        matrix (ee.ndarray): The utility matrix
        predictions (Any): Rating for a movie by a user to be made
        knn (int): The number of nearest neighbors to be used in the algorithm
        return_dict (Any): A return dictonnary used to return the results of a Process
        pid (int): The id of the current process
        global_average (float): The average rating of the train set
        users_bias (Dict): A dictionary containing the biases of all users.
        movies_bias (Dict): A dictionary containing the biases of all movies.
    """
    result = []
    for index in range(len(predictions)):
        userp = int(predictions[index, 0])-1
        moviep = int(predictions[index, 1])-1
        sim = get_similarities(userp, moviep, matrix)
        sim.sort(key=operator.itemgetter(1), reverse=True)
        simk = np.asarray(sim[0:knn])
        rating = 0
        sum = 0
        for i in simk:
            m = int(i[0])
            s = i[1]
            if(np.isnan(s)):
                continue
            sum = sum + s
            rating = rating + s * (matrix[m, userp] - get_baseline_estimate(m, userp, global_average, users_bias, movies_bias))
        if (sum == 0):
            rating = get_baseline_estimate(moviep, userp, global_average, users_bias, movies_bias)
        else:
            rating = get_baseline_estimate(moviep, userp, global_average, users_bias, movies_bias) + (rating / sum)
        result.append((index+1, rating))
    return_dict[pid] = result
    print("finished job in process "+str(pid+1))

The **get_similarities** function finds the similarities of an entry to other entries.

In [7]:
def get_similarities(user_id: int, movie_id: int, matrix: np.ndarray) -> List[float]:
    """Calculate the similarity of the current movie to other movies.

    Args:
        user_id (int): The index of the user
        movie_id (int): The index of the movie
        matrix (ee.ndarray): The utility matrix

    Returns:
        List[float]: The list of similarities
    """
    sim = list()
    a = matrix[movie_id, :]
    anan = np.argwhere(np.isnan(a)).transpose().flatten()
    for i in range(matrix.shape[0]):
        if(i == movie_id or np.isnan(matrix[i, user_id])): 
            continue
        b = matrix[i, :]
        bnan = np.argwhere(np.isnan(b)).transpose().flatten()
        delx = np.unique(np.concatenate((anan, bnan)))
        ax = np.delete(a, delx)
        bx = np.delete(b, delx)
        if(len(ax) <= 1 or len(bx) <= 1): 
            continue
        corr, p_value = pearsonr(ax, bx)
        # Handling of PearsonRConstantInputWarning 
        if np.isnan(corr):
            corr = 0.0
        sim.append((i, corr))
    return sim

The **get_baseline_estimate** function is used to get the baseline estimate (Global + Local) for a particular entry. This is done to get better estimates.

In [8]:
def get_baseline_estimate(movie_id: int, user_id: int, global_average: float, users_bias: Dict, movies_bias: Dict) -> float:
    """Calculate the baseline estimate for a particular movie and user

    Args:
        movie_id (int): The index of the movie
        user_id (int): The index of the user
        global_average (float): The average rating of the train set
        users_bias (Dict): A dictionary containing the biases of all users.
        movies_bias (Dict): A dictionary containing the biases of all movies.

    Returns:
        float: the baseline estimate
    """    
    return global_average+users_bias[user_id]+movies_bias[movie_id]

The **get_biases** function will be used to calculate the biases for movies and users. This is needed to add the local effects for the final predicted ratings.

In [9]:
def get_biases(utility_matrix: np.ndarray, global_average: float) -> (Dict, Dict):
    """Calculate biases for movies and users.

    Args:
        utility_matrix (ee.ndarray): The utility matrix
        global_average (float): The average rating for the train set

    Returns:
        (Dict, Dict): Dictionaries for user and movie biases
    """    
    # Calculate the user biases
    users_bias = dict()
    for i in range(utility_matrix.shape[1]):
        m = np.nanmean(utility_matrix[:, i])
        # Handling of Mean of empty slice runtime warning
        if(np.isnan(m)):
            users_bias[i] = 0.0
        else: 
            users_bias[i] = m - global_average
    
    # Calculate the movies biases
    movies_bias = dict()
    for i in range(utility_matrix.shape[0]):
        m = np.nanmean(utility_matrix[i, :])
        # Handling of Mean of empty slice runtime warning
        if(np.isnan(m)):
            movies_bias[i] = 0.0
        else:
            movies_bias[i] = m - global_average

    return (users_bias, movies_bias)

The **calculate_RMSE** function is used to calculate the Root Mean Squared Error which is used to find the accuracy of the algorithm on test data set.

In [10]:
def calculate_RMSE(results: List[float], prediction_set: np.ndarray) -> float:
    """Calculate the RMSE between the results and prediction set.

    Args:
        results (List[float]): The list of predicted results
        test_set (ee.ndarray): The test set 

    Returns:
        float: The RMSE between the results and test set
    """
    expected = prediction_set[:, 2].flatten()
    assert len(expected) == len(results)
    return math.sqrt(mean_squared_error(expected, results))

This is the main function. Multiprocessing will be used to speed up the computation by creating multiple processes. The number of processes is defined in the config file.

**Note:** Multiprocessing doesn't work well with Jupyter Notebook when running on Windows. There might be some issues when the program is executed on Windows. It is preferred to use Linux to run this notebook. Setting the number of processes to 1 could also help.

In [11]:
if __name__ == '__main__':
    
    # Number of processes
    num_processes = config["number_processes"]
    # Number of neighbours
    num_neighbours = config["number_neighbours"]
    (number_users, number_movies, max_ratings, max_timestamp) = np.max(train_set, axis=0)
    number_predictions = len(test_set)
    number_ratings = len(train_set)
    global_average = train_set.mean(axis=0)[2]
    # test_set = test_set[0:1000, :]
    (users_bias, movies_bias) = get_biases(utility_matrix, global_average)

    start_time = time.time()
    # Create multiple chunks so each chunk can be assigned to a different process
    chunks = np.array_split(test_set, num_processes)
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    processes = []
    # Create the multiple processes
    for i in range(num_processes):
        print("starting process: "+str(i+1))
        process = Process(target=predict_nearestneighbor_mp, args=(utility_matrix, chunks[i], num_neighbours, return_dict, i, global_average, users_bias, movies_bias))
        processes.append(process)
        process.start()
    for j in processes:
        j.join()
    
    # Retrieve the results from the multiple processes.
    pr1 = return_dict.items()
    pr1.sort(key=operator.itemgetter(0), reverse=False)
    pr2 = list()
    for i in pr1:
        pr2.append(i[1])
    flattened_list = [y for x in pr2 for y in x]
    print(flattened_list)
    results = [p[1] for p in flattened_list]
    total_time = (time.time() - start_time)
    rmse = calculate_RMSE(results, test_set)
    print("--- " + str(total_time) + " seconds ---")
    print("--- rmse: " + str(rmse) + " ---")
    logger.save(total_time, rmse)

<ipython-input-9-1facae9144f8>:24: RuntimeWarning: Mean of empty slice
  m = np.nanmean(utility_matrix[i, :])


starting process: 1
starting process: 2
starting process: 3
starting process: 4
starting process: 5


/home/rami/.local/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/rami/.local/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


starting process: 6
starting process: 7


/home/rami/.local/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


starting process: 8


/home/rami/.local/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/rami/.local/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/rami/.local/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/rami/.local/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/rami/.local/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input 

finished job in process 7
finished job in process 8
finished job in process 5
finished job in process 1
finished job in process 4
finished job in process 6
finished job in process 2
finished job in process 3
--- 1037.5483438968658 seconds ---
--- rmse: 31.390324472551338 ---
log: nearest_neighbour, 1037.5483438968658, 31.390324472551338, 8, , 9, 0.005, 20, 12, 0.05, False, 5

